<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [23]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,sklearn,pandas
# %watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
# %watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal
import numpy as np

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try:
        x = x.strip()
        if x[0] == '/' or x[0:2] == './':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)

def get_logger(name, level=logging.DEBUG, filepath=None, backup_count=-1, console=True, mp=False):
    logger = logging.getLogger(name)
    logger.handlers.clear()
    if isinstance(level, str):
        if level in ('D', 'DEBUG', 'd', 'debug'):
            level = logging.DEBUG 
        elif level in ('I', 'INFO', 'i', 'info'):
            level = logging.INFO
        elif level in ('E', 'ERROR', 'e', 'error'):
            level = logging.ERROR
        else:
            level = logging.WARNING
    logger.setLevel(level)
    handlers = []
    #  %(filename)s
    formatter = logging.Formatter('%(asctime)s - %(funcName)s:%(lineno)d - %(name)s - %(levelname)s - %(message)s')
    if console:
        handlers.append(logging.StreamHandler())
    if filepath:
        if backup_count > 0:
            filelog = logging.handlers.TimedRotatingFileHandler(
                filename=filepath,
                when='D',
                backupCount=backup_count,
                encoding='utf-8')
        else:  
            filelog = logging.FileHandler(filepath)
        handlers.append(filelog)
        
    if mp: # multiprocessing
        handlers = [MultiProcessingHandler(name, handlers)]
        
    for handler in handlers:
        handler.setLevel(level)
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    return logger

In [1]:
import logging
import multiprocessing
import sys
import threading
import traceback
import queue

class MultiProcessingHandler(logging.Handler):
    def __init__(self, name, handlers=None):
        super(MultiProcessingHandler, self).__init__()
        if handlers is None or len(handlers) == 0:
            handlers = [logging.StreamHandler()]
        self.handlers = handlers
        self.queue = multiprocessing.Queue(-1)
        self._is_closed = False
        self._receive_thread = threading.Thread(target=self._receive, name=name)
        self._receive_thread.daemon = True
        self._receive_thread.start()
        
    def setLevel(self, level):
        super(MultiProcessingHandler, self).setLevel(level)
        for handler in self.handlers:
            handler.setLevel(level)

    def setFormatter(self, fmt):
        super(MultiProcessingHandler, self).setFormatter(fmt)
        for handler in self.handlers:
            handler.setFormatter(fmt)

    def _receive(self):        
        while True:
            try:
                if self._is_closed and self.queue.empty():
                    break
                record = self.queue.get(timeout=0.2)
                for handler in self.handlers:
                    handler.emit(record)
            except (KeyboardInterrupt, SystemExit):
                raise
            except (BrokenPipeError, EOFError):
                break
            except queue.Empty:
                pass  # This periodically checks if the logger is closed.
            except:
                traceback.print_exc(file=sys.stderr)

        self.queue.close()
        self.queue.join_thread()

    def _send(self, s):
        self.queue.put_nowait(s)

    def _format_record(self, record):
        if record.args:
            record.msg = record.msg % record.args
            record.args = None
        if record.exc_info:
            self.format(record)
            record.exc_info = None

        return record

    def emit(self, record):
        try:
            s = self._format_record(record)
            self._send(s)
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            self.handleError(record)

    def close(self):
        if not self._is_closed:
            self._is_closed = True
            self._receive_thread.join(5.0)  # Waits for receive queue to empty.
            for handler in self.handlers:
                handler.close()
            super(MultiProcessingHandler, self).close()

In [3]:
from multiprocessing import Process, Queue, Value, Array, Pipe, Lock
from enum import Enum
from abc import ABC, abstractmethod
import inspect
import sys

class State(Enum):
    STOP = "S_STOP"
    SHUTDOWN = "S_SHUTDOWN"
    SHUTDOWN_LAST = "S_SHUTDOWN_LAST"
    
    
class ICallable(ABC):
    
    @classmethod
    def init(self):
        pass
    
    @abstractmethod
    def __call__(self, x):
        ...
        
    @classmethod
    def shutdown(self):
        pass
    
    
class Task(object):
    def __init__(self, id, fn, input_queue, output_queue, multiplicity):
        self.id = id
        self.fn = fn
        self.input_queue = input_queue
        self.output_queue = output_queue
        self.multiplicity = multiplicity

    def start(self):
        self.process = Process(target=self.main_loop, args=(self.input_queue, self.output_queue))
        self.process.start()

    def main_loop(self, input_queue, output_queue):
        self.input_queue = input_queue
        self.output_queue = output_queue

        try:
            if hasattr(self.fn, "init"):
                self.fn.init()

            while True:
                x = self.input_queue.get()
                if  x == State.SHUTDOWN:
                    break
                if x == State.SHUTDOWN_LAST:
                    self.output_queue.put(State.STOP)
                    break
                if x == State.STOP: 
                    for i in range(self.multiplicity-1):
                        self.input_queue.put(State.SHUTDOWN)
                    self.input_queue.put(State.SHUTDOWN_LAST)
                    continue

                result = self.fn(x)
                if inspect.isgenerator(result):
                    for x in result:
                        if x == State.STOP: 
                            self.input_queue.put(State.STOP)
                            break
                        self.output_queue.put(x)
                else:
                    if result == State.STOP:
                        self.input_queue.put(State.STOP)
                    else:
                        self.output_queue.put(result)

            if hasattr(self.fn, "shutdown"):
                self.fn.shutdown()

        except KeyboardInterrupt:
            pass
        except Exception:
            print("For {}".format(self.fn))
            raise

    
class TaskPipeline(object):
    def __init__(self):
        self.tasks = []
        self.input_queue = Queue(1)
        self.output_queue = Queue(1)
        self.nextId = 1

    def run(self, arg = None):
        for task in self.tasks:
            task.start()
        self.input_queue.put(arg)
        while True:
            x = self.output_queue.get()
            if x == State.STOP:
                break

    def add(self, func, fan_out=1):
        input_queue = self.input_queue
        output_queue = self.output_queue
        if len(self.tasks):
            input_queue = Queue(1)
            self.tasks[-1].output_queue = input_queue

        for i in range(fan_out):
            task = Task(self.nextId, func, input_queue, output_queue, fan_out)
            self.nextId += 1
            self.tasks.append(task)
    

In [ ]:
def input_func(x):
    for i in range(x):
        yield i
    yield State.STOP
    
def output_func(x):
    yield x * 2

class ResultTest(ICallable):
    def __init__(self, arg):
        self.arg = arg
        
    def __call__(self, x):
        print('result', x)
        
pipe = TaskPipeline()
pipe.add(input_func)
pipe.add(output_func)
pipe.add(ResultTest('test'))
pipe.run(5)

In [4]:
class ShmNumpy(object):
    def __init__(self, dtype, shape):
        self.shm_size = int(np.prod(shape))
        self.mp_array = Array(dtype, self.shm_size, lock=Lock())
        self.np_array = np.frombuffer(self.mp_array.get_obj(), dtype=dtype)
        self.dsize, self.ddim = Value('I', self.shm_size), Value('I', 3)
        self.dshape = Array('I', shape)
    
    @property
    def data(self):
        return self.np_array[:self.dsize.value].reshape(self.dshape[:self.ddim.value])
    
    @data.setter
    def data(self, d):
        assert self.shm_size >= d.size
        self.np_array[:d.size] = d.ravel()
        self.ddim.value = len(d.shape)
        self.dshape[:len(d.shape)] = d.shape
        self.dsize.value = d.size
        
    @property
    def shape(self):
        return self.dshape
    
    def lock(self):
        self.mp_array.acquire()
        
    def unlock(self):
        self.mp_array.release()

class TaskV2(object):
    def __init__(self, id, func, input_pipe, output_pipe, inshms=[], outshms=[]):
        self.id = id
        self.fn = func
        self.input_pipe, self.output_pipe = input_pipe, output_pipe
        self.inshms, self.outshms = inshms, outshms
        
    def inshms(self):
        return self.inshms
        
    def outshms(self):
        return self.outshms

    def start(self):
        self.process = Process(target=self.main_loop, args=(self.input_pipe, self.output_pipe))
        self.process.start()

    def main_loop(self, input_pipe, output_pipe):
        try:
            if hasattr(self.fn, "init"):
                self.fn.init()

            while True:
                x = self.input_pipe.recv()
                # sys.stderr.write(f'<{x}>\n')
                if x == State.STOP: 
                    self.output_pipe.send(State.STOP)
                    break

                result = self.fn(x, *self.inshms, *self.outshms)
                if inspect.isgenerator(result):
                    for x in result:
                        self.output_pipe.send(x)
                else:
                    self.output_pipe.send(result)

            if hasattr(self.fn, "shutdown"):
                self.fn.shutdown()

        except KeyboardInterrupt:
            pass
        except Exception:
            print("For {}".format(self.fn))
            raise
        
class TaskPipelineV2(object):
    def __init__(self):
        self.tasks = []
        self.nextId = 1
        self.input_pipe, self.output_pipe = Pipe()
        
    def run(self, x=None):
        for task in self.tasks:
            task.start()
        self.input_pipe.send(x)
        while True:
            x = self.output_pipe.recv()
            if x == State.STOP:
                break
       
    def add(self, fn, shms=[]):
        input_pipe, output_pipe = Pipe()
        
        inshms, outshms = [], shms
        if len(self.tasks) > 0:
            inshms = self.tasks[-1].outshms
            
        self.tasks.append(TaskV2(self.nextId, fn, self.output_pipe, input_pipe, inshms, outshms))
        self.nextId += 1
        self.output_pipe = output_pipe

In [25]:
log = get_logger('mp-loggering', level='debug', mp=True)
def input_func(x, shm_frame):
    for i in range(x):
        shm_frame.lock()
        shm_frame.data = np.ones(shm_frame.shape) * i
        log.info(i)
        y = {'a': i}
        yield  y
    yield State.STOP
    
def output_func(x, shm_frame):
    y = x['a'] * 2
    log.info(f'{shm_frame.data[0, 0, :]}')
    shm_frame.unlock()
    yield y

class ResultTest(ICallable):
    def __init__(self, arg):
        self.arg = arg
        
    def __call__(self, x):
        log.info(f'result: {x}')
        
pipe = TaskPipelineV2()
pipe.add(input_func, shms=[ShmNumpy('I', (640, 352, 3))])
pipe.add(output_func)
pipe.add(ResultTest('test'))
pipe.run(10)

2022-02-16 21:32:08,453 - input_func:6 - mp-loggering - INFO - 0
2022-02-16 21:32:08,455 - output_func:13 - mp-loggering - INFO - [0 0 0]
2022-02-16 21:32:08,457 - __call__:22 - mp-loggering - INFO - result: 0
2022-02-16 21:32:08,461 - input_func:6 - mp-loggering - INFO - 1
2022-02-16 21:32:08,461 - output_func:13 - mp-loggering - INFO - [1 1 1]
2022-02-16 21:32:08,461 - __call__:22 - mp-loggering - INFO - result: 2
2022-02-16 21:32:08,463 - input_func:6 - mp-loggering - INFO - 2
2022-02-16 21:32:08,463 - output_func:13 - mp-loggering - INFO - [2 2 2]
2022-02-16 21:32:08,463 - __call__:22 - mp-loggering - INFO - result: 4
2022-02-16 21:32:08,465 - input_func:6 - mp-loggering - INFO - 3
2022-02-16 21:32:08,465 - output_func:13 - mp-loggering - INFO - [3 3 3]
2022-02-16 21:32:08,465 - __call__:22 - mp-loggering - INFO - result: 6
2022-02-16 21:32:08,466 - input_func:6 - mp-loggering - INFO - 4
2022-02-16 21:32:08,467 - output_func:13 - mp-loggering - INFO - [4 4 4]
2022-02-16 21:32:08,46

In [ ]:
c = b.ravel()

In [ ]:
d.size

In [ ]:
c[:24] = d

In [ ]:
m = Array("I", 100, lock=Lock())

In [ ]:
np.frombuffer(m.get_obj())[:10]
i

In [ ]:
input_pipe, output_pipe = Pipe()

In [ ]:
input_pipe.send('1')
input_pipe.send('1')
input_pipe.send('1')
output_pipe.recv()

In [ ]:
output_pipe.recv()